# Create clone IDs for each patient.

Create clone IDs across all specimens (i.e. all timepoints, mixing peak and non-peak samples -- and including all replicates of those timepoints) from a patient.

Clone IDs are not unique across patients

In [1]:
from pathlib import Path
from typing import List
import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed

In [2]:
from malid import config, helpers, etl, get_v_sequence, io, logger
from malid.datamodels import GeneLocus, healthy_label
from malid.sample_sequences import sample_sequences
from malid.trained_model_wrappers import ConvergentClusterClassifier

In [3]:
n_jobs = 40

# get specimen filepaths from specimen metadata list

## covid samples

In [4]:
covid_specimens = pd.read_csv(
    config.paths.metadata_dir
    / "generated.external_cohorts.covid19_bcr.specimen_metadata_extra.tsv",
    sep="\t",
)
covid_specimens

,specimen_label,participant_label,timepoint,is_peak
0,5f21e814e1adeb2edc12613c,Kim_A,11,False
1,5f21e814e1adeb2edc12613d,Kim_A,17,True
2,5f21e815e1adeb2edc12613e,Kim_A,45,False
3,5f21e815e1adeb2edc12613f,Kim_B,10,False
4,5f21e815e1adeb2edc126140,Kim_B,19,True
...,...,...,...,...
91,6028546736266613226-242ac116-0001-012,Montague_8,32,False
92,5977479575117173226-242ac116-0001-012,Montague_8,37,False
93,5994186997898613226-242ac116-0001-012,Montague_8,37,False
94,6148719921208693226-242ac116-0001-012,Montague_9,5,False


In [5]:
covid_specimens.shape

(96, 4)

In [6]:
participant_df = pd.read_csv(
    config.paths.metadata_dir
    / "generated.external_cohorts.covid19_bcr.participant_metadata.tsv",
    sep="\t",
)
participant_df

,participant_label,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name
0,Kim_A,PRJNA648677,A,M,55.0,Asian,Case,Covid19,Kim
1,Kim_B,PRJNA648677,B,M,55.0,Asian,Case,Covid19,Kim
2,Kim_C,PRJNA648677,C,F,53.0,Asian,Case,Covid19,Kim
3,Kim_D,PRJNA648677,D,M,24.0,Asian,Case,Covid19,Kim
4,Kim_E,PRJNA648677,E,M,48.0,Asian,Case,Covid19,Kim
5,Kim_F,PRJNA648677,F,F,40.0,Asian,Case,Covid19,Kim
6,Kim_G,PRJNA648677,G,F,59.0,Asian,Case,Covid19,Kim
7,Montague_1,PRJNA645245,1,F,62.0,NaN,Mild,Covid19,Montague
8,Montague_2,PRJNA645245,2,F,37.0,NaN,Mild,Covid19,Montague
9,Montague_3,PRJNA645245,3,M,47.0,NaN,Moderate,Covid19,Montague


In [7]:
covid_specimens = pd.merge(
    covid_specimens, participant_df, how="left", validate="m:1", on="participant_label"
)
covid_specimens

,specimen_label,participant_label,timepoint,is_peak,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name
0,5f21e814e1adeb2edc12613c,Kim_A,11,False,PRJNA648677,A,M,55.0,Asian,Case,Covid19,Kim
1,5f21e814e1adeb2edc12613d,Kim_A,17,True,PRJNA648677,A,M,55.0,Asian,Case,Covid19,Kim
2,5f21e815e1adeb2edc12613e,Kim_A,45,False,PRJNA648677,A,M,55.0,Asian,Case,Covid19,Kim
3,5f21e815e1adeb2edc12613f,Kim_B,10,False,PRJNA648677,B,M,55.0,Asian,Case,Covid19,Kim
4,5f21e815e1adeb2edc126140,Kim_B,19,True,PRJNA648677,B,M,55.0,Asian,Case,Covid19,Kim
...,...,...,...,...,...,...,...,...,...,...,...,...
91,6028546736266613226-242ac116-0001-012,Montague_8,32,False,PRJNA645245,8,M,37.0,NaN,Moderate,Covid19,Montague
92,5977479575117173226-242ac116-0001-012,Montague_8,37,False,PRJNA645245,8,M,37.0,NaN,Moderate,Covid19,Montague
93,5994186997898613226-242ac116-0001-012,Montague_8,37,False,PRJNA645245,8,M,37.0,NaN,Moderate,Covid19,Montague
94,6148719921208693226-242ac116-0001-012,Montague_9,5,False,PRJNA645245,9,F,52.0,NaN,Moderate,Covid19,Montague


In [8]:
covid_specimens.shape

(96, 12)

In [9]:
covid_specimens["disease_subtype"] = (
    covid_specimens["disease"]
    + " - "
    + covid_specimens["study_name"]
    + covid_specimens["is_peak"].replace({True: "", False: " (non-peak)"})
)
covid_specimens["gene_locus"] = GeneLocus.BCR.name
covid_specimens

,specimen_label,participant_label,timepoint,is_peak,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name,gene_locus
0,5f21e814e1adeb2edc12613c,Kim_A,11,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR
1,5f21e814e1adeb2edc12613d,Kim_A,17,True,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR
2,5f21e815e1adeb2edc12613e,Kim_A,45,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR
3,5f21e815e1adeb2edc12613f,Kim_B,10,False,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR
4,5f21e815e1adeb2edc126140,Kim_B,19,True,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,6028546736266613226-242ac116-0001-012,Montague_8,32,False,PRJNA645245,8,M,37.0,NaN,Covid19 - Montague (non-peak),Covid19,Montague,BCR
92,5977479575117173226-242ac116-0001-012,Montague_8,37,False,PRJNA645245,8,M,37.0,NaN,Covid19 - Montague (non-peak),Covid19,Montague,BCR
93,5994186997898613226-242ac116-0001-012,Montague_8,37,False,PRJNA645245,8,M,37.0,NaN,Covid19 - Montague (non-peak),Covid19,Montague,BCR
94,6148719921208693226-242ac116-0001-012,Montague_9,5,False,PRJNA645245,9,F,52.0,NaN,Covid19 - Montague (non-peak),Covid19,Montague,BCR


In [10]:
covid_specimens["fname"] = covid_specimens.apply(
    lambda row: config.paths.external_raw_data
    / "covid_external_as_part_tables"
    / f"exported.part_table.{row['specimen_label']}.tsv",
    axis=1,
)
covid_specimens.head()

,specimen_label,participant_label,timepoint,is_peak,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name,gene_locus,fname
0,5f21e814e1adeb2edc12613c,Kim_A,11,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
1,5f21e814e1adeb2edc12613d,Kim_A,17,True,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
2,5f21e815e1adeb2edc12613e,Kim_A,45,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
3,5f21e815e1adeb2edc12613f,Kim_B,10,False,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
4,5f21e815e1adeb2edc126140,Kim_B,19,True,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...


## healthy specimens

In [11]:
healthy_specimens = pd.read_csv(
    config.paths.metadata_dir
    / "generated.external_cohorts.healthy_bcr.participant_metadata.tsv",
    sep="\t",
)

# process peak samples only
healthy_specimens = healthy_specimens[healthy_specimens["is_peak"] == True]

healthy_specimens["disease_subtype"] = (
    healthy_specimens["disease"]
    + " - "
    + healthy_specimens["study_name"]
    + healthy_specimens["is_peak"].replace({True: "", False: " (non-peak)"})
)

healthy_specimens["gene_locus"] = GeneLocus.BCR.name

healthy_specimens

,participant_label,specimen_label,study_name,disease,is_peak,timepoint,age,sex,ethnicity,ethnicity_condensed,disease_subtype,gene_locus
0,D103,D103_1,Briney,Healthy/Background,True,0,25,M,Caucasian,Caucasian,Healthy/Background - Briney,BCR
1,326797,326797_1,Briney,Healthy/Background,True,0,21,F,Caucasian,Caucasian,Healthy/Background - Briney,BCR
2,327059,327059_1,Briney,Healthy/Background,True,0,26,M,African American / Caucasian,NaN,Healthy/Background - Briney,BCR
3,326650,326650_1,Briney,Healthy/Background,True,0,18,F,Caucasian,Caucasian,Healthy/Background - Briney,BCR
4,326737,326737_1,Briney,Healthy/Background,True,0,29,M,Caucasian,Caucasian,Healthy/Background - Briney,BCR
5,326780,326780_1,Briney,Healthy/Background,True,0,29,M,Caucasian,Caucasian,Healthy/Background - Briney,BCR
6,316188,316188_1,Briney,Healthy/Background,True,0,30,F,African American,African,Healthy/Background - Briney,BCR


In [12]:
healthy_specimens["fname"] = healthy_specimens.apply(
    lambda row: config.paths.external_raw_data
    / "briney_healthy_as_part_tables"
    / f"exported.part_table.{row['specimen_label']}.tsv",
    axis=1,
)
healthy_specimens.head()

,participant_label,specimen_label,study_name,disease,is_peak,timepoint,age,sex,ethnicity,ethnicity_condensed,disease_subtype,gene_locus,fname
0,D103,D103_1,Briney,Healthy/Background,True,0,25,M,Caucasian,Caucasian,Healthy/Background - Briney,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
1,326797,326797_1,Briney,Healthy/Background,True,0,21,F,Caucasian,Caucasian,Healthy/Background - Briney,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
2,327059,327059_1,Briney,Healthy/Background,True,0,26,M,African American / Caucasian,NaN,Healthy/Background - Briney,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
3,326650,326650_1,Briney,Healthy/Background,True,0,18,F,Caucasian,Caucasian,Healthy/Background - Briney,BCR,/users/maximz/code/boyd-immune-repertoire-clas...
4,326737,326737_1,Briney,Healthy/Background,True,0,29,M,Caucasian,Caucasian,Healthy/Background - Briney,BCR,/users/maximz/code/boyd-immune-repertoire-clas...


## healthy TCR specimens

In [13]:
tcr_healthy_specimens = pd.read_csv(
    config.paths.metadata_dir
    / "generated.external_cohorts.healthy_tcr_britanova.participant_metadata.tsv",
    sep="\t",
).assign(
    is_peak=True,
    gene_locus=GeneLocus.TCR.name,
)
tcr_healthy_specimens["fname"] = tcr_healthy_specimens["specimen_label"].apply(
    lambda specimen_label: config.paths.external_raw_data
    / "chudakov_aging"
    / f"{specimen_label}.txt.gz"
)

tcr_healthy_specimens

,specimen_label,sex,age,ethnicity_condensed,participant_label,disease,study_name,disease_subtype,is_peak,gene_locus,fname
0,A3-i101,F,36,Caucasian,p1,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
1,A3-i102,F,43,Caucasian,p2,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
2,A3-i106,F,43,Caucasian,p3,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
3,A3-i107,F,39,Caucasian,p4,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
4,A3-i110,F,34,Caucasian,p5,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
5,A2-i138,F,74,Caucasian,p15,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
6,A2-i139,M,75,Caucasian,p16,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
7,A2-i140,F,73,Caucasian,p17,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
8,A2-i141,M,71,Caucasian,p18,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
9,A4-i101,M,36,Caucasian,p19,Healthy/Background,Britanova,Healthy/Background - Britanova,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...


## Covid TCR specimens

In [14]:
tcr_covid_specimens = pd.read_csv(
    config.paths.metadata_dir
    / "generated.external_cohorts.covid_tcr_shomuradova.participant_metadata.tsv",
    sep="\t",
).assign(
    is_peak=True,
    gene_locus=GeneLocus.TCR.name,
)
tcr_covid_specimens["fname"] = tcr_covid_specimens["specimen_label"].apply(
    lambda specimen_label: config.paths.external_raw_data
    / "shomuradova"
    / f"split.{specimen_label}.tsv"
)

tcr_covid_specimens

,specimen_label,participant_label,disease,study_name,disease_subtype,age,sex,ethnicity_condensed,is_peak,gene_locus,fname
0,5f07aa8839579433171763b4,p1437,Covid19,Shomuradova,Covid19 - mild,28,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
1,5f07aa8939579433171763b7,p1445,Covid19,Shomuradova,Covid19 - mild,32,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
2,5f07aa8a39579433171763ba,p1473,Covid19,Shomuradova,Covid19 - mild,31,F,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
3,5f07aa8c39579433171763c0,p1489,Covid19,Shomuradova,Covid19 - mild,27,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
4,6047f702136a6d924982945c,p1434,Covid19,Shomuradova,Covid19 - mild,28,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
5,6047f703136a6d924982945f,p1448,Covid19,Shomuradova,Covid19 - moderate/severe,37,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
6,6047f704136a6d9249829462,p1449,Covid19,Shomuradova,Covid19 - mild,34,F,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
7,6047f704136a6d9249829465,p1465,Covid19,Shomuradova,Covid19 - moderate/severe,19,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
8,6047f706136a6d924982946b,p1480,Covid19,Shomuradova,Covid19 - moderate/severe,29,M,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...
9,6047f707136a6d924982946e,p1481,Covid19,Shomuradova,Covid19 - moderate/severe,30,F,Caucasian,True,TCR,/users/maximz/code/boyd-immune-repertoire-clas...


## Adaptive healthy TCR specimens

In [15]:
adaptive_tcr_healthy_specimens = pd.DataFrame(
    {"fname": (config.paths.external_raw_data / "emerson").glob("*.tsv")}
).assign(
    disease=healthy_label,
    study_name="Emerson",
    is_peak=True,
    gene_locus=GeneLocus.TCR.name,
    # Flag that these are a different platform than what we expect
    different_platform=True,
)

# extract specimen label from filename
adaptive_tcr_healthy_specimens = adaptive_tcr_healthy_specimens.assign(
    specimen_label=adaptive_tcr_healthy_specimens["fname"].apply(lambda path: path.stem)
)
assert not adaptive_tcr_healthy_specimens["specimen_label"].duplicated().any()
# participants are 1:1 with specimens
adaptive_tcr_healthy_specimens["participant_label"] = adaptive_tcr_healthy_specimens[
    "specimen_label"
]

# TODO: add sex, age, ethnicity

adaptive_tcr_healthy_specimens["disease_subtype"] = (
    adaptive_tcr_healthy_specimens["disease"]
    + " - "
    + adaptive_tcr_healthy_specimens["study_name"]
    + adaptive_tcr_healthy_specimens["is_peak"].replace(
        {True: "", False: " (non-peak)"}
    )
)

adaptive_tcr_healthy_specimens

,fname,disease,study_name,is_peak,gene_locus,different_platform,specimen_label,participant_label,disease_subtype
0,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,P00311,P00311,Healthy/Background - Emerson
1,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,Keck0119_MC1,Keck0119_MC1,Healthy/Background - Emerson
2,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,P00104,P00104,Healthy/Background - Emerson
3,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,P00511,P00511,Healthy/Background - Emerson
4,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,Keck0013_MC1,Keck0013_MC1,Healthy/Background - Emerson
...,...,...,...,...,...,...,...,...,...
781,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,P00331,P00331,Healthy/Background - Emerson
782,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,Keck0086_MC1,Keck0086_MC1,Healthy/Background - Emerson
783,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,P00183,P00183,Healthy/Background - Emerson
784,/users/maximz/code/boyd-immune-repertoire-clas...,Healthy/Background,Emerson,True,TCR,True,P00121,P00121,Healthy/Background - Emerson


## Adaptive Covid TCR specimens

In [16]:
adaptive_tcr_covid_specimens = pd.read_csv(
    config.paths.metadata_dir
    / "generated.external_cohorts.adaptive_covid_tcr.specimens.tsv",
    sep="\t",
).assign(
    study_name="ImmuneCode",
    is_peak=True,
    gene_locus=GeneLocus.TCR.name,
    # Flag that these are a different platform than what we expect
    different_platform=True,
)
adaptive_tcr_covid_specimens["fname"] = adaptive_tcr_covid_specimens[
    "specimen_label"
].apply(
    lambda specimen_label: config.paths.external_raw_data
    / "immunecode"
    / "reps"
    / "ImmuneCODE-Review-002"
    / f"{specimen_label}.tsv"
)

adaptive_tcr_covid_specimens

,participant_label,specimen_label,disease,disease_subtype,age,sex,ethnicity_condensed,study_name,is_peak,gene_locus,different_platform,fname
0,ImmuneCode-190921,860011232_TCRB,Covid19,COVID-19-HUniv12Oct - Hospitalized,53,F,Hispanic/Latino,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
1,ImmuneCode-026,INCOV026-AC-3_TCRB,Covid19,COVID-19-ISB,33,M,Hispanic/Latino,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
2,ImmuneCode-321977,860011116_TCRB,Covid19,COVID-19-HUniv12Oct - Hospitalized,52,F,Caucasian,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
3,ImmuneCode-087,INCOV087-BL-3_TCRB,Covid19,COVID-19-ISB,56,M,Hispanic/Latino,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
4,ImmuneCode-0000051,BS-EQ-0014-T2-replacement_TCRB,Covid19,COVID-19-NIH/NIAID - Hospitalized,55,M,Caucasian,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
...,...,...,...,...,...,...,...,...,...,...,...,...
88,ImmuneCode-ADIRP0001958,ADIRP0001958_TCRB,Covid19,COVID-19-Adaptive,49,M,Caucasian,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
89,ImmuneCode-775827,860011106_TCRB,Covid19,COVID-19-HUniv12Oct - Hospitalized,57,F,Caucasian,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
90,ImmuneCode-0000446,BS-GIGI_10-replacement_TCRB,Covid19,COVID-19-NIH/NIAID - Hospitalized,75,M,Caucasian,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...
91,ImmuneCode-0000160,BS-EQ-25-T1_BS-GIGI-71-replacement_TCRB,Covid19,COVID-19-NIH/NIAID - Hospitalized - ICU,72,M,Caucasian,ImmuneCode,True,TCR,True,/users/maximz/code/boyd-immune-repertoire-clas...


## Instructions for adding more

If we have external cohorts that are BCR+TCR, we should have one row per locus per specimen.

Set a `specimen_label_by_locus` column that is the globally-unique specimen label tailored to a particular locus, e.g. `$SPECIMENLABEL-IGH` or `$SPECIMENLABEL-TRB` format.

And set a `specimen_label` column that is equivalent across different-loci rows for that specimen.

The row's `gene_locus` column should be set to the locus of that row (must be the name of a valid `GeneLocus` enum value), and the `fname` column should be set to the path to the file containing the data for that locus.

## merge

In [17]:
all_specimens = pd.concat(
    [
        covid_specimens,
        healthy_specimens,
        tcr_healthy_specimens,
        tcr_covid_specimens,
        adaptive_tcr_healthy_specimens,
        adaptive_tcr_covid_specimens,
    ],
    axis=0,
)

# fillna
all_specimens["different_platform"].fillna(False, inplace=True)

all_specimens

,specimen_label,participant_label,timepoint,is_peak,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name,gene_locus,fname,ethnicity,different_platform
0,5f21e814e1adeb2edc12613c,Kim_A,11.0,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False
1,5f21e814e1adeb2edc12613d,Kim_A,17.0,True,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False
2,5f21e815e1adeb2edc12613e,Kim_A,45.0,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False
3,5f21e815e1adeb2edc12613f,Kim_B,10.0,False,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False
4,5f21e815e1adeb2edc126140,Kim_B,19.0,True,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,ADIRP0001958_TCRB,ImmuneCode-ADIRP0001958,NaN,True,NaN,NaN,M,49.0,Caucasian,COVID-19-Adaptive,Covid19,ImmuneCode,TCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,True
89,860011106_TCRB,ImmuneCode-775827,NaN,True,NaN,NaN,F,57.0,Caucasian,COVID-19-HUniv12Oct - Hospitalized,Covid19,ImmuneCode,TCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,True
90,BS-GIGI_10-replacement_TCRB,ImmuneCode-0000446,NaN,True,NaN,NaN,M,75.0,Caucasian,COVID-19-NIH/NIAID - Hospitalized,Covid19,ImmuneCode,TCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,True
91,BS-EQ-25-T1_BS-GIGI-71-replacement_TCRB,ImmuneCode-0000160,NaN,True,NaN,NaN,M,72.0,Caucasian,COVID-19-NIH/NIAID - Hospitalized - ICU,Covid19,ImmuneCode,TCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,True


In [18]:
# Fillna for cohorts that are single-locus
if "specimen_label_by_locus" not in all_specimens:
    # in case we had no BCR+TCR combined cohorts that set this field already
    all_specimens["specimen_label_by_locus"] = all_specimens["specimen_label"]
else:
    all_specimens["specimen_label_by_locus"].fillna(
        all_specimens["specimen_label"], inplace=True
    )

In [19]:
# make sure input fnames exist
assert all_specimens["fname"].apply(os.path.exists).all()

In [20]:
all_specimens.shape

(1038, 17)

In [21]:
# Set age_group column as well, just as in assemble_etl_metadata
all_specimens["age"].describe()

count    252.000000
mean      50.861111
std       17.042265
min        8.000000
25%       37.000000
50%       52.000000
75%       62.000000
max       89.000000
Name: age, dtype: float64

In [22]:
all_specimens["age_group"] = pd.cut(
    all_specimens["age"],
    bins=[0, 20, 30, 40, 50, 60, 70, 80, 100],
    labels=["<20", "20-30", "30-40", "40-50", "50-60", "60-70", "70-80", "80+"],
    right=False,
)
all_specimens["age_group"].value_counts()

50-60    62
60-70    41
30-40    37
40-50    37
20-30    33
70-80    29
80+      10
<20       3
Name: age_group, dtype: int64

In [23]:
all_specimens["age_group"].cat.categories

Index(['<20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80+'], dtype='object')

In [24]:
all_specimens["age"].isna().value_counts()

True     786
False    252
Name: age, dtype: int64

In [25]:
all_specimens["age_group"].isna().value_counts()

True     786
False    252
Name: age_group, dtype: int64

In [26]:
for age_group, grp in all_specimens.groupby("age_group"):
    print(age_group, grp["age"].min(), grp["age"].max())

<20 8.0 19.0
20-30 20.0 29.0
30-40 30.0 39.0
40-50 40.0 49.0
50-60 50.0 59.0
60-70 60.0 68.0
70-80 70.0 79.0
80+ 80.0 89.0


In [27]:
# Just as in assemble_etl_metadata:
# Null out "age_group" column for extreme ages with small sample size.

# Note that we are not getting rid of these specimens altogether,
# but marking age_group NaN will disable their use for demographics-controlling models

orig_shapes = all_specimens.shape[0], all_specimens["age_group"].isna().sum()
mask = all_specimens["age_group"].isin(["80+"])
all_specimens.loc[mask, "age_group"] = np.nan
new_shapes = all_specimens.shape[0], all_specimens["age_group"].isna().sum()

# sanity checks:
# - we did not drop any specimens
assert orig_shapes[0] == new_shapes[0]
# - but we did null out some age_group entries
assert orig_shapes[1] < new_shapes[1]
# - we nulled out the right amount
assert new_shapes[1] - orig_shapes[1] == mask.sum()

In [28]:
# export for later processing
all_specimens.drop(["fname"], axis=1).to_csv(
    config.paths.metadata_dir / "generated.external_cohorts.all_specimens.tsv",
    sep="\t",
    index=None,
)

In [29]:
# confirm all specimen labels are unique within each locus (may have one BCR and one TCR line per specimen)
# TODO: in the future, allow for replicates of each specimen
assert not all_specimens["specimen_label_by_locus"].duplicated().any()
for locus, grp in all_specimens.groupby("gene_locus"):
    assert not grp["specimen_label"].duplicated().any()

In [30]:
# Which specimens are in multiple loci?
all_specimens[all_specimens["specimen_label"].duplicated(keep=False)]

,specimen_label,participant_label,timepoint,is_peak,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name,gene_locus,fname,ethnicity,different_platform,specimen_label_by_locus,age_group


In [31]:
all_specimens["study_name"].value_counts()

Emerson        786
ImmuneCode      93
Montague        80
Britanova       39
Shomuradova     17
Kim             16
Briney           7
Name: study_name, dtype: int64

In [32]:
all_specimens["disease"].value_counts()

Healthy/Background    832
Covid19               206
Name: disease, dtype: int64

In [33]:
all_specimens["gene_locus"].value_counts()

TCR    935
BCR    103
Name: gene_locus, dtype: int64

In [34]:
all_specimens["disease_subtype"].value_counts()

Healthy/Background - Emerson                786
Covid19 - Montague (non-peak)                52
Healthy/Background - Britanova               39
COVID-19-NIH/NIAID - Hospitalized            39
Covid19 - Montague                           28
COVID-19-HUniv12Oct - Hospitalized           26
Covid19 - mild                               10
Covid19 - Kim (non-peak)                      9
COVID-19-ISB                                  9
Covid19 - Kim                                 7
Healthy/Background - Briney                   7
Covid19 - moderate/severe                     7
COVID-19-NIH/NIAID - Hospitalized - ICU       7
COVID-19-HUniv12Oct - Hospitalized - ICU      6
COVID-19-Adaptive                             5
COVID-19-HUniv12Oct                           1
Name: disease_subtype, dtype: int64

In [35]:
all_specimens["different_platform"].value_counts()

True     879
False    159
Name: different_platform, dtype: int64

In [36]:
all_specimens.groupby(["different_platform", "disease_subtype"]).size()

different_platform  disease_subtype                         
False               Covid19 - Kim                                 7
                    Covid19 - Kim (non-peak)                      9
                    Covid19 - Montague                           28
                    Covid19 - Montague (non-peak)                52
                    Covid19 - mild                               10
                    Covid19 - moderate/severe                     7
                    Healthy/Background - Briney                   7
                    Healthy/Background - Britanova               39
True                COVID-19-Adaptive                             5
                    COVID-19-HUniv12Oct                           1
                    COVID-19-HUniv12Oct - Hospitalized           26
                    COVID-19-HUniv12Oct - Hospitalized - ICU      6
                    COVID-19-ISB                                  9
                    COVID-19-NIH/NIAID - Hospitalized  

In [37]:
for demographics_column in ["age", "age_group", "sex", "ethnicity_condensed"]:
    print(demographics_column)
    print(all_specimens[demographics_column].value_counts())
    print(all_specimens[demographics_column].isna().value_counts())
    print()

age
37.0    15
55.0    13
51.0    11
62.0    11
48.0     9
        ..
32.0     1
22.0     1
74.0     1
18.0     1
42.0     1
Name: age, Length: 67, dtype: int64
True     786
False    252
Name: age, dtype: int64

age_group
50-60    62
60-70    41
30-40    37
40-50    37
20-30    33
70-80    29
<20       3
80+       0
Name: age_group, dtype: int64
True     796
False    242
Name: age_group, dtype: int64

sex
M    135
F    117
Name: sex, dtype: int64
True     786
False    252
Name: sex, dtype: int64

ethnicity_condensed
Caucasian          137
Asian               18
Hispanic/Latino     12
African              1
Name: ethnicity_condensed, dtype: int64
True     870
False    168
Name: ethnicity_condensed, dtype: int64



In [38]:
# TODO: Separate all above into a separate "compile all metadata" notebook.

# process specimen, drop duplicates, and cluster each group (-> clones)


In [39]:
def process_specimen(
    fname: Path, gene_locus: GeneLocus, study_name: str, specimen_label: str
):
    # defensive cast
    fname = Path(fname)

    # each specimen is one "repertoire_id"
    df = pd.read_csv(
        fname,
        sep="\t",
        # Solve "DtypeWarning: Columns (9,17,25,33) have mixed types. Specify dtype option on import or set low_memory=False."
        dtype={
            "pre_seq_nt_q": "object",
            "pre_seq_nt_v": "object",
            "pre_seq_nt_d": "object",
            "pre_seq_nt_j": "object",
        },
        # special N/A values for Adaptive data
        na_values=["no data", "unknown", "unresolved"],
    )

    # Rename columns
    if "specimen_label" not in df.columns:
        if "sample_name" in df.columns:
            df.rename(
                columns={
                    "sample_name": "specimen_label",
                },
                inplace=True,
            )
        elif "repertoire_id" in df.columns:
            df.rename(
                columns={
                    "repertoire_id": "specimen_label",
                },
                inplace=True,
            )
        else:
            df = df.assign(specimen_label=specimen_label)

    # confirm only one specimen included here
    if not (df["specimen_label"] == specimen_label).all():
        raise ValueError(
            f"Processing specimen {specimen_label}, but specimen_label column in {fname} does not match this."
        )

    # Recognize sample type
    if study_name == "Britanova" and gene_locus == GeneLocus.TCR:
        # Special case for this study:
        # No raw sequences available to run IgBlast ourselves.

        # Rename columns
        df.rename(
            columns={
                "v": "v_gene",
                "j": "j_gene",
                "cdr3nt": "cdr3_seq_nt_q",
                "cdr3aa": "cdr3_seq_aa_q",
                "count": "num_reads",
            },
            inplace=True,
        )

        # Pulling in CDR1 and CDR2 for TCRB data in etl._compute_columns expects v_segment to be set with allele info.
        # etl._compute_columns guarantees that each V gene has a CDR1 and CDR2 available for its dominant *01 allele.
        df["v_segment"] = df["v_gene"] + "*01"
        df["j_segment"] = df["j_gene"] + "*01"

        # Trim CDR3 AA: remove ends
        # and replace field that's entirely space (or empty) with NaN
        # (maybe we should be trimming cdr3-nt too, but that's only used for clustering within patient to set clone IDs, so it doesn't matter)
        df.dropna(subset=["cdr3_seq_aa_q"], inplace=True)
        df["cdr3_seq_aa_q"] = (
            df["cdr3_seq_aa_q"]
            .str.slice(start=1, stop=-1)
            .replace(r"^\s*$", np.nan, regex=True)
        )
        df.dropna(subset=["cdr3_seq_aa_q"], inplace=True)

        # Mark these all as productive TCRB
        df["productive"] = True
        df["extracted_isotype"] = "TCRB"

    else:
        # We have run sequences through our IgBlast.
        # Note that our legacy IgBlast parser uses the location of our Boydlab primers to parse the CDR3 sequence
        # So if you run shorter sequences like Adaptive seuqences, we won't parse IgBlast's CDR3 calls, but we will still get V/J calls.
        is_adaptive = "bio_identity" in df.columns

        if study_name in ["Briney", "Kim", "Montague"]:
            # Special case: iReceptor / VDJserver / Briney studies that went through legacy Postgres-based annotations,
            # i.e. igblast parses already merged in.

            # TODO: update runbook for these to be consistent with new schema where we merge igblast parses on the fly,
            # then eliminate this special case.

            # Igblast parse exported from Postgres database: cast to bool to be compatible
            df["productive"] = df["productive"].replace({"t": True, "f": False})

            # get v_sequence (same way we produce v_sequence in internal pipeline's sort script)
            (
                df["v_sequence"],
                df["d_sequence"],
                df["j_sequence"],
            ) = get_v_sequence.complete_sequences(df)

        else:
            if is_adaptive:
                # Adaptive, reprocessed through our IgBlast
                # See https://www.adaptivebiotech.com/wp-content/uploads/2019/07/MRK-00342_immunoSEQ_TechNote_DataExport_WEB_REV.pdf

                # Our IgBlast gives some different V gene calls, but generally doesn't provide CDR3 calls for these short sequences.
                # That's because our parser looks for the location of our primers.
                # We'll use the V/J gene and productive calls from our IgBlast, while using Adaptive's CDR3 call.

                if "cdr3_rearrangement" in df.columns:
                    df.rename(
                        columns={"cdr3_rearrangement": "cdr3_seq_nt_q"}, inplace=True
                    )
                elif "rearrangement" in df.columns:
                    # this is the entire detected rearrangement, not just cdr3, but will suffice for finding nucleotide uniques
                    df.rename(columns={"rearrangement": "cdr3_seq_nt_q"}, inplace=True)
                else:
                    raise ValueError(
                        f"No nucleotide rearrangement column found for {specimen_label}"
                    )

                # Define whether productive from Adaptive metadata
                df["productive"] = df["frame_type"] == "In"

                # Count copies using templates or seq_reads field, whichever is available
                if not df["templates"].isna().any():
                    df["num_reads"] = df["templates"]
                elif not df["seq_reads"].isna().any():
                    df["num_reads"] = df["seq_reads"]
                else:
                    raise ValueError(
                        f"Could not choose templates/seq_reads column from {specimen_label}"
                    )

                # Also extract Adaptive's V/J gene calls (-> "original_*" columns) and CDR3 AA call.
                # Follow tcrdist3's import_adaptive_file pattern (https://github.com/kmayerb/tcrdist3/blob/55d56fa621ec19b25a31ee1a3e61ef60e2575837/tcrdist/adpt_funcs.py#L24):
                # Don't just parse v_gene and j_gene; parse bio_identity instead.

                # Per https://tcrdist3.readthedocs.io/en/latest/adaptive.html:
                # "Adaptive’s output files can contain gene-level names within the ‘bioidentity’ field like TCRBV15-X, when there is ambiguity about the gene-level assignment."
                # Format example:
                # {'v_gene': 'unresolved',
                # 'v_gene_ties': 'TCRBV12-03/12-04,TCRBV12-04',
                # 'bio_identity': 'CATSAISSNQPQHF+TCRBV12-X+TCRBJ01-05'}
                df[["cdr3_seq_aa_q", "original_v_segment", "original_j_segment"]] = df[
                    "bio_identity"
                ].str.split("+", expand=True)

                # Trim CDR3: remove ends
                # and replace field that's entirely space (or empty) with NaN
                df.dropna(subset=["cdr3_seq_aa_q"], inplace=True)
                df["cdr3_seq_aa_q"] = (
                    df["cdr3_seq_aa_q"]
                    .str.slice(start=1, stop=-1)
                    .replace(r"^\s*$", np.nan, regex=True)
                )

            # Merge in igblast parses to get better V/J gene calls.
            parse_fnames = list(
                (fname.parent / "igblast_splits").glob(
                    f"split.{specimen_label}.*.parsed.tsv"
                )
            )
            if len(parse_fnames) == 0:
                raise ValueError(
                    f"No igblast parse files found for {specimen_label} from {study_name}"
                )
            df_parse = pd.concat(
                [pd.read_csv(fname, sep="\t") for fname in parse_fnames], axis=0
            )

            # extract fasta ID
            df_parse[["specimen_label", "rownum"]] = df_parse["id"].str.split(
                "|", expand=True
            )
            df_parse["rownum"] = df_parse["rownum"].astype(int)
            assert not df_parse["rownum"].duplicated().any()

            # For now we are assuming df and df_parse both have exactly one specimen
            assert (df_parse["specimen_label"] == specimen_label).all()

            if not is_adaptive:
                # get v_sequence (same way we produce v_sequence in internal pipeline's sort script)
                # this will be used to compute v_mut for BCR
                (
                    df_parse["v_sequence"],
                    df_parse["d_sequence"],
                    df_parse["j_sequence"],
                ) = get_v_sequence.complete_sequences(df_parse)
            else:
                # Adaptive->IgBlast reprocessing does not have the necessary sequence info for us,
                # because our Igblast-output parser fails to extract some sequence regions since Adaptive sequences are shorter.
                df_parse["v_sequence"] = pd.Series(dtype="str")
                df_parse["d_sequence"] = pd.Series(dtype="str")
                df_parse["j_sequence"] = pd.Series(dtype="str")

            orig_shape = df.shape
            df = pd.merge(
                df.rename(
                    # store original metadata as "original_*" columns
                    columns={
                        "productive": "original_productive",
                        "v_gene": "original_v_gene",
                        "j_gene": "original_j_gene",
                        "cdr3_seq_nt_q": "original_cdr3_seq_nt_q",
                        "cdr3_seq_aa_q": "original_cdr3_seq_aa_q",
                        # possible original Adaptive metadata:
                        "v_gene_ties": "original_v_gene_ties",
                        "j_gene_ties": "original_j_gene_ties",
                    }
                ),
                df_parse.set_index("rownum")[
                    [
                        "v_segment",
                        "j_segment",
                        "productive",
                        "v_sequence",
                        "d_sequence",
                        "j_sequence",
                    ]
                    + (
                        # Adaptive IgBlast rerun does not have sequences when interpreted by our parser,
                        # because Adaptive's sequences are shorter than ours
                        [
                            "cdr1_seq_aa_q",
                            "cdr2_seq_aa_q",
                            "cdr3_seq_nt_q",
                            "cdr3_seq_aa_q",
                        ]
                        if not is_adaptive
                        else []
                    )
                ],
                left_index=True,
                right_index=True,
                how="inner",
                validate="1:1",
            )
            assert df.shape[0] == min(orig_shape[0], df_parse.shape[0])

            # For Adaptive data, since CDR3 information was missing in IgBlast parses,
            # set cdr3_seq_nt_q and cdr3_seq_aa_q to the original values.
            if is_adaptive:
                df["cdr3_seq_nt_q"] = df["original_cdr3_seq_nt_q"]
                df["cdr3_seq_aa_q"] = df["original_cdr3_seq_aa_q"]
                # Also set other missing columns for consistency.
                # Not sure if these are true assumptions, but we will subset to TRBV V genes later, so should be ok
                df["locus"] = "TRB"
                df["extracted_isotype"] = "TCRB"

        # drop the external study's sequence_id (keep the int primary key instead)
        df = df.drop(columns="sequence_id", errors="ignore")

        # rename columns for consistency
        df.rename(
            columns={
                "c_call": "extracted_isotype",
                "id": "sequence_id",
            },
            inplace=True,
        )
        if gene_locus == GeneLocus.TCR:
            # sanity check
            if not (df["locus"] == "TRB").all():
                raise ValueError(
                    f"Locus field was not TRB for {specimen_label} ({study_name})"
                )
            # set isotype flag if iReceptor c_call is blank
            df["extracted_isotype"].fillna("TCRB", inplace=True)

        if study_name == "Montague":
            # extracted_isotype is not provided, but we know these are IgG
            if not all(df["extracted_isotype"].isna()):
                raise ValueError("We expect no isotype calls for Montague")
            df["extracted_isotype"].fillna("IGHG", inplace=True)

    # replace extracted_isotype values to have consistent prefix: IgG -> IGHG, IgA -> IGHA, etc.
    df["extracted_isotype"] = df["extracted_isotype"].str.replace(
        "^Ig", "IGH", regex=True
    )

    # filter
    df_orig_shape = df.shape
    df = df.loc[(~pd.isnull(df["extracted_isotype"])) & (df["productive"] == True)]
    if df.shape[0] == 0:
        raise ValueError(f"Filtering failed for {specimen_label} from {study_name}")

    # compute important columns
    # note that this converts v_segment, j_segment (with alleles) to v_gene, j_gene columns (no alleles).
    df = etl._compute_columns(df=df, gene_locus=gene_locus)

    # Replace indistinguishable TRBV gene names with the version that we use in our data.
    df["v_gene"] = df["v_gene"].replace(io.v_gene_renames)

    # since we are going to call clones by nucleotide sequences here rather than in the usual bioinformatics pipeline,
    # let's also preprocess the NT characters.
    # (note for Adaptive data: we had to remove the prefix/suffix from the CDR3 AA sequence.
    # we aren't bothering to do that with the nucleotide sequence, since we are just using that to set clone IDs
    # it's not being passed to the downstream language model, so it doesn't have to be consistent with the rest of our data.)
    df["cdr3_seq_nt_q_trim"] = etl._trim_sequences(df["cdr3_seq_nt_q"])
    df.dropna(
        subset=[
            "cdr3_seq_nt_q_trim",
        ],
        how="any",
        inplace=True,
    )

    # get trimmed lengths
    df["cdr3_nt_sequence_trim_len"] = df["cdr3_seq_nt_q_trim"].str.len()

    # Now that everything has gone through our IgBlast, these filters should be no-ops:
    # Downselect only to V genes that are in our standard dataset
    invalid_v_genes = set(df["v_gene"].unique()) - set(
        helpers.all_observed_v_genes()[gene_locus]
    )
    if len(invalid_v_genes) > 0:
        logger.warning(
            f"Dropping V genes from {specimen_label} ({study_name}) that aren't in our standard data: {invalid_v_genes}"
        )
        df = df.loc[df["v_gene"].isin(helpers.all_observed_v_genes()[gene_locus])]

    # And downselect only to J genes that are in our standard dataset
    invalid_j_genes = set(df["j_gene"].unique()) - set(
        helpers.all_observed_j_genes()[gene_locus]
    )
    if len(invalid_j_genes) > 0:
        logger.warning(
            f"Dropping J genes from {specimen_label} ({study_name}) that aren't in our standard data: {invalid_j_genes}"
        )
        df = df.loc[df["j_gene"].isin(helpers.all_observed_j_genes()[gene_locus])]

    # make each row a single unique VDJ sequence - drop duplicates
    # save number of reads to a column called `num_reads`, to measure clonal expansion later
    dedupe_cols = [
        "specimen_label",
        "extracted_isotype",
        "v_gene",
        "j_gene",
        "cdr1_seq_aa_q_trim",
        "cdr2_seq_aa_q_trim",
        "cdr3_seq_aa_q_trim",
        "cdr3_seq_nt_q_trim",
        "productive",
    ]
    if "num_reads" in df.columns:
        read_counts = (
            df.groupby(dedupe_cols, observed=True)["num_reads"].sum().reset_index()
        )
    else:
        read_counts = (
            df.groupby(dedupe_cols, observed=True).size().reset_index(name="num_reads")
        )
    df.drop_duplicates(
        subset=dedupe_cols,
        keep="first",
        inplace=True,
    )
    # sanity check
    if not all(df.groupby(dedupe_cols, observed=True).size() == 1):
        raise ValueError(f"Deduplicate failed for {specimen_label} from {study_name}")
    expected_shape = df.shape
    # merge in the num_reads column, replacing (dropping) the existing num_reads column if it exists
    df = pd.merge(
        df.drop(columns="num_reads", errors="ignore"),
        read_counts,
        how="left",
        on=dedupe_cols,
        validate="1:1",
    )
    if df.shape[0] != expected_shape[0]:
        raise ValueError(
            f"Merge post dedupe failed for {specimen_label} from {study_name}"
        )

    return df, (fname, df_orig_shape, df.shape, gene_locus)

In [40]:
all_specimens.head()

,specimen_label,participant_label,timepoint,is_peak,study_id,patient_id_within_study,sex,age,ethnicity_condensed,disease_subtype,disease,study_name,gene_locus,fname,ethnicity,different_platform,specimen_label_by_locus,age_group
0,5f21e814e1adeb2edc12613c,Kim_A,11.0,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False,5f21e814e1adeb2edc12613c,50-60
1,5f21e814e1adeb2edc12613d,Kim_A,17.0,True,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False,5f21e814e1adeb2edc12613d,50-60
2,5f21e815e1adeb2edc12613e,Kim_A,45.0,False,PRJNA648677,A,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False,5f21e815e1adeb2edc12613e,50-60
3,5f21e815e1adeb2edc12613f,Kim_B,10.0,False,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim (non-peak),Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False,5f21e815e1adeb2edc12613f,50-60
4,5f21e815e1adeb2edc126140,Kim_B,19.0,True,PRJNA648677,B,M,55.0,Asian,Covid19 - Kim,Covid19,Kim,BCR,/users/maximz/code/boyd-immune-repertoire-clas...,NaN,False,5f21e815e1adeb2edc126140,50-60


In [41]:
def run_on_all_specimens_from_a_person(participant_label, specimens):
    dfs = []
    info_per_specimen = []

    # This isn't true yet, but later on we might have BCR and TCR versions of the same specimen.
    # Get combined gene locus flag for all gene loci used in this group of specimens
    gene_loci = [GeneLocus[name] for name in specimens["gene_locus"].unique()]
    # combine flags
    gene_loci = GeneLocus.combine_flags_list_into_single_multiflag_value(gene_loci)
    GeneLocus.validate(gene_loci)

    # Load each specimen on its own. We expect one entry for BCR and another for TCR.
    for _, specimen in specimens.iterrows():
        gene_locus_for_this_specimen = GeneLocus[specimen["gene_locus"]]
        df, annotation = process_specimen(
            fname=specimen["fname"],
            gene_locus=gene_locus_for_this_specimen,
            study_name=specimen["study_name"],
            # note: passing in specimen label by locus. because on disk separated by (and named by) locus.
            specimen_label=specimen["specimen_label_by_locus"],
        )

        # assign metadata
        df = df.assign(
            participant_label=specimen["participant_label"],
            timepoint=specimen["timepoint"],
            is_peak=specimen["is_peak"],
            disease=specimen["disease"],
            disease_subtype=specimen["disease_subtype"],
            # at this point, rename specimen_label from current value (specimen_label_by_locus) to non-locus value (specimen_label)
            specimen_label=specimen["specimen_label"],
        )

        dfs.append((gene_locus_for_this_specimen, df))
        info_per_specimen.append(annotation)

    # cluster all specimens from this patient staple sequences from the specimens together.
    # but run clustering separately on different gene loci, because of different distance criteria.
    dfs_clustered_for_each_gene_locus = [
        ConvergentClusterClassifier._cluster_training_set(
            df=pd.concat([df for (gl, df) in dfs if gl == gene_locus], axis=0),
            sequence_identity_threshold=config.sequence_identity_thresholds.call_nucleotide_clones_with_patient[
                gene_locus
            ],
            validate_same_fold=False,
            higher_order_group_columns=[
                "v_gene",
                "j_gene",
                "cdr3_nt_sequence_trim_len",
            ],
            sequence_column="cdr3_seq_nt_q_trim",
            inplace=True,
        )
        for gene_locus in gene_loci
    ]

    # we can then combine clustering results across gene loci because V genes will be different (and V gene is included in cluster ID).
    df = pd.concat(dfs_clustered_for_each_gene_locus, axis=0)

    # Express global clone ID as string instead
    df.drop("global_resulting_cluster_ID", axis=1, inplace=True)
    df["igh_or_tcrb_clone_id"] = (
        df[
            [
                "specimen_label",
                "v_gene",
                "j_gene",
                "cdr3_nt_sequence_trim_len",
                "cluster_id_within_clustering_group",
            ]
        ]
        .astype(str)
        .apply(tuple, axis=1)
        .apply("|".join)
    )

    # Clustering also created a num_clone_members=1 column
    # Drop that so we can set it properly in sample_sequences
    df.drop("num_clone_members", axis=1, inplace=True)

    # Report total number of clones
    logger.info(
        f"Participant {participant_label} ({gene_loci}) has {df['igh_or_tcrb_clone_id'].nunique()} unique clones from specimens: {info_per_specimen}"
    )

    # Sample clones from each specimen,
    # with filters like dropping low-SHM naive B cells.
    # df may be empty after this
    df = pd.concat(
        [
            sample_sequences(specimen_df, required_gene_loci=gene_loci)
            for specimen_label, specimen_df in df.groupby("specimen_label")
        ],
        axis=0,
    )

    if df.shape[0] != 0:
        # Determine output filename: one file per external cohort participant.
        # Gene loci are combined into one file. They can be separated by the isotype column.
        fname_out = (
            config.paths.external_processed_data / f"{participant_label}.parquet"
        )
        if fname_out.exists():
            logger.warning(f"Path already exists, overwriting: {fname_out}")

        # Report number of sampled sequences
        logger.info(
            f"Participant {participant_label} ({gene_loci}) has {df.shape[0]} sampled sequences -> {fname_out}."
        )

        # Write
        df.to_parquet(fname_out, index=None)
        return fname_out
    else:
        logger.warning(
            f"Participant {participant_label} ({gene_loci}) had no sampled sequences. Skipping."
        )
        return None

In [42]:
# run on all specimens from same patient (combine all timepoints - even if mixed peak/non-peak status)
fnames_output = Parallel(n_jobs=n_jobs, backend="multiprocessing")(
    delayed(run_on_all_specimens_from_a_person)(
        participant_label=participant_label, specimens=specimens
    )
    for participant_label, specimens in all_specimens.groupby(
        ["participant_label"], observed=True
    )
)

2022-12-28 17:44:38,120 - py.warnings - WARNING - /tmp/ipykernel_1465441/1673077630.py:8: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(



2022-12-28 17:44:41,628 - py.warnings - WARNING - /tmp/ipykernel_1465441/1673077630.py:8: DtypeWarning: Columns (39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(



2022-12-28 17:44:44,355 - py.warnings - WARNING - /tmp/ipykernel_1465441/1673077630.py:8: DtypeWarning: Columns (39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(



2022-12-28 17:44:44,708 - py.warnings - WARNING - /tmp/ipykernel_1465441/1673077630.py:8: DtypeWarning: Columns (39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(



2022-12-28 17:44:51,739 - py.warnings - WARNING - /tmp/ipykernel_1465441/1673077630.py:8: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(



...

2022-12-28 19:45:33,819 - assign_clone_ids.ipynb - WARNING - Dropping V genes from A6-I201ob (Britanova) that aren't in our standard data: {'TRBV21-1', 'TRBV23-1'}


2022-12-28 19:45:34,546 - assign_clone_ids.ipynb - INFO - Participant p36 (GeneLocus.TCR) has 886393 sampled sequences -> /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/external_cohorts_part_tables/p36.parquet.


2022-12-28 19:46:03,164 - assign_clone_ids.ipynb - INFO - Participant p53 (GeneLocus.TCR) has 924067 sampled sequences -> /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/external_cohorts_part_tables/p53.parquet.


2022-12-28 19:49:58,155 - assign_clone_ids.ipynb - INFO - Participant p76 (GeneLocus.TCR) has 2092047 unique clones from specimens: [(PosixPath('/users/maximz/code/boyd-immune-repertoire-classification/data/external_cohorts/raw_data/chudakov_aging/A6-I200ob.txt.gz'), (1812062, 16), (1723172, 26), <GeneLocus.TCR: 2>), (PosixPath('/users/maximz/code/boyd-immune-repertoire-classification/data/external_cohorts/raw_data/chudakov_aging/A6-I201ob.txt.gz'), (758985, 16), (721976, 26), <GeneLocus.TCR: 2>)]


2022-12-28 19:52:07,129 - assign_clone_ids.ipynb - INFO - Participant p76 (GeneLocus.TCR) has 2090120 sampled sequences -> /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/external_cohorts_part_tables/p76.parquet.


In [43]:
len(fnames_output)

965

In [44]:
# drop null returns
fnames_output = [f for f in fnames_output if f is not None]
len(fnames_output)

945

In [45]:
# Many, but not all, output fnames will exist now
len(fnames_output), all_specimens.shape[0]

(945, 1038)

In [46]:
pd.read_parquet(
    fnames_output[0],
).head()

,sequence_id,participant_label,specimen_label,extracted_isotype,v_segment,j_segment,cdr1_seq_aa_q,cdr2_seq_aa_q,cdr3_seq_aa_q,pre_seq_nt_q,...,cdr3_nt_sequence_trim_len,num_reads,timepoint,is_peak,disease,disease_subtype,cluster_id_within_clustering_group,igh_or_tcrb_clone_id,total_clone_num_reads,num_clone_members
0,6028499,326650,326650_1,IGHM,IGHV4-31*03,IGHJ6*02,G G S I S S G G Y Y,I Y Y S G S T,A R D A W V R G N Y G M D V,None,...,42,1,0.0,True,Healthy/Background,Healthy/Background - Briney,62.0,326650_1|IGHV4-31|IGHJ6|42|62.0,1,1
1,6028543,326650,326650_1,IGHG3,IGHV1-8*01,IGHJ4*02,G Y T F T S Y D,M N P N S G N T,A R G T T A E N,None,...,24,1,0.0,True,Healthy/Background,Healthy/Background - Briney,41.0,326650_1|IGHV1-8|IGHJ4|24|41.0,1,1
448206,7528081,326650,326650_1,IGHG1,IGHV3-7*01,IGHJ4*02,G F T F S N Y W,I R Q D G S E K,A R S T A G L D Y,None,...,27,2,0.0,True,Healthy/Background,Healthy/Background - Briney,621.0,326650_1|IGHV3-7|IGHJ4|27|621.0,9,7
3,6028638,326650,326650_1,IGHG4,IGHV4-59*01,IGHJ4*02,G G S I S S Y Y,I Y Y S G S T,A R G Y A E E Q H F D Y,None,...,36,1,0.0,True,Healthy/Background,Healthy/Background - Briney,3922.0,326650_1|IGHV4-59|IGHJ4|36|3922.0,1,1
4,6028661,326650,326650_1,IGHG1,IGHV1-46*01,IGHJ3*02,G Y T F T S Y Y,I N P S G G S T,A S Q S P D A F D I,None,...,30,1,0.0,True,Healthy/Background,Healthy/Background - Briney,20.0,326650_1|IGHV1-46|IGHJ3|30|20.0,1,1


In [47]:
pd.read_parquet(
    fnames_output[0],
).columns

Index(['sequence_id', 'participant_label', 'specimen_label',
       'extracted_isotype', 'v_segment', 'j_segment', 'cdr1_seq_aa_q',
       'cdr2_seq_aa_q', 'cdr3_seq_aa_q', 'pre_seq_nt_q', 'fr1_seq_nt_q',
       'cdr1_seq_nt_q', 'fr2_seq_nt_q', 'cdr2_seq_nt_q', 'fr3_seq_nt_q',
       'cdr3_seq_nt_q', 'post_seq_nt_q', 'pre_seq_nt_v', 'fr1_seq_nt_v',
       'cdr1_seq_nt_v', 'fr2_seq_nt_v', 'cdr2_seq_nt_v', 'fr3_seq_nt_v',
       'cdr3_seq_nt_v', 'post_seq_nt_v', 'pre_seq_nt_d', 'fr1_seq_nt_d',
       'cdr1_seq_nt_d', 'fr2_seq_nt_d', 'cdr2_seq_nt_d', 'fr3_seq_nt_d',
       'cdr3_seq_nt_d', 'post_seq_nt_d', 'pre_seq_nt_j', 'fr1_seq_nt_j',
       'cdr1_seq_nt_j', 'fr2_seq_nt_j', 'cdr2_seq_nt_j', 'fr3_seq_nt_j',
       'cdr3_seq_nt_j', 'post_seq_nt_j', 'productive', 'v_sequence',
       'd_sequence', 'j_sequence', 'isotype_supergroup', 'v_gene', 'j_gene',
       'cdr1_seq_aa_q_trim', 'cdr2_seq_aa_q_trim', 'cdr3_seq_aa_q_trim',
       'cdr3_aa_sequence_trim_len', 'v_mut', 'cdr3_seq_nt_q_trim'